In [ ]:
from nltk.ccg.lexicon import CCGLexicon, Token, augParseCategory
from nltk.ccg.chart import CCGChart,CCGLeafEdge,BinaryCombinatorRule,CCGEdge,CCGChartParser
from nltk.ccg.chart import compute_semantics,printCCGDerivation
from nltk.ccg.combinator import *
from nltk.tree import Tree
from nltk.sem.logic import Expression
from numbers import Number
import pandas as pd
import random
from functools import reduce

# 1. Creating the Lexicon

The lexicon has first been created in a libreoffice spreadsheet, and is under the file `ccg.ods`.
For each word of the corpus, we gave one, two or three categories in the columns Cat0, Cat1 and Cat2. We also indicated the «classical french grammar name» of the word in order to help us find the catégories. We also did group some words together (with slashes), for example donne/mange have the same catégories, same for le/la/un/mon/ses.

In order to do the rest of the project, we also added a *weight* column for each word group/category couple, and we also added a *semantics* column for each word/category couple, so we could assignate a semantics to be read by the program afterwards.

This allowed us a clean reading, editing and tuning of every parameter of our lexicon.

The spreadsheet is then directly read into the program

## 2.a Robustness of the grammar
Our grammar is really simple, as it has really few catégories. However, every category has been highly tuned for its purpose, so very few agrammatical sentences slip through. However we did not take into account tenses, genre and plurals by choice, so some phrases that are parsed may be agrammatical in that regard - for instance, "il le attrappe" or "est (elle souhaite ses fromage)" or "le souris" are parsed. But for what we consider a successful parse, it is a strict grammar.

## 2.b Ambiguity
The main cause of ambiguity is that there is very few catégories, therefore there is a lot of derivation trees going to the same goal. For example, because we do not differentiate *(méchant chat) noir* and *méchant (chat noir)* because both correspond to a reduction (pN pN pN -> pN pN -> pN). Because we have so few catégories, we don't have things like «adjectives order» that would fix the order in which those trees are parsed.

# 4. Implementation of the CKY algorithm

This set of functions compiles into the function `bestTree` that returns the best derivation tree of set of tokens, the lexer we extracted from the spreadsheet, and the set of rules.

We define the weight associated to each reduction rule.
`rweight(rule)` should return the weight associated to the rule, using its string representation (i.e. the name of the rule)

In [ ]:
valz = {
    '>' : 0.8,
    '<' : 0.8,
    '<B' : 0.7,
    '>B' : 0.7,
    '<Bx' : 0.6,
    '<Sx' : 0.6,
    '<S' : 0.65,
    '>Bx' : 0.6,
    '>Sx' : 0.6,
    '>S' : 0.65
}
def rweight(rule):
    s = rule.__str__()
    if s in valz:
        return valz[s]
    else:
        print("Unknown rule",s)
        return 1.0

`weightedParse` implements the CKY algorithm, based on the implementation in the nltk library.
We take the weight from the weighted lexicon for the leafs, and we compute it using the formula for each reduction rule.
$$ w_{node} = \phi_r \times w_{child1} \times w_{child2}$$

In [ ]:
# Implements the CYK algorithm, code partly taken from nltk
def weightedParse(tokens, lex, rules):
    chart = CCGChart(list(tokens))
    
    # Initialize leaf edges.
    for index in range(chart.num_leaves()):
        for token in lex.categories(chart.leaf(index)):
            new_edge = CCGLeafEdge(index, token, chart.leaf(index))
            new_edge.weight = token.weight()
            chart.insert(new_edge, ())

    # Select a span for the new edges
    for span in range(2, chart.num_leaves() + 1):
        for start in range(0, chart.num_leaves() - span + 1):
            
            # edges[s] is the best edge generating the category s
            edges = dict()
            
            # Try all possible pairs of edges that could generate
            # an edge for that span
            for part in range(1, span):
                lstart = start
                mid = start + part
                rend = start + span
                
                # For every pair of edges in (lstart,mid) / (mid,rend).
                # They could be multiple edges if they are two categories on the same span
                for left in chart.select(span=(lstart, mid)):
                    for right in chart.select(span=(mid, rend)):
                        
                        # Generate all possible combinations of the two edges
                        for rule in rules:
                            
                            # Can we apply the rule
                            if rule.can_combine(left.categ(), right.categ()):
                                
                                # If so, we create a (potential) new edge for each new category
                                for res in rule.combine(left.categ(), right.categ()):
                                    
                                    # res is the new category
                                    edge = CCGEdge(
                                        span=(left.start(), right.end()),
                                        categ=res,
                                        rule=rule,
                                    )
                                    # We compute the weight of the edge
                                    edge.weight = rweight(rule) * left.weight * right.weight
                                    # And we log the information of where the triple comes from
                                    edge.triple = (rule,left,right)
                                    # We remember the heaviest edge for the specific category
                                    if not(res in edges and edges[res].weight<=edge.weight):
                                        edges[res] = edge
                        # end for rule loop
                    # end for right loop
                # end for left loop
            # end for part loop
            # We add for each category the heaviest edge found
            for cat in edges:
                chart.insert(edges[cat], (edges[cat].triple[1], edges[cat].triple[2]))
    
    # We can return the chart we created
    return chart

In [ ]:
def wp_to_tree(edge, wChart):
    """
        This functions takes the biggest edge of a chart and the chart, and send back
        the heaviest tree, that is, the one that generated the weightedParse function.
    """
    if isinstance(edge,CCGLeafEdge):
        word = Tree(edge.token(), [wChart._tokens[edge.start()]])
        leaf = Tree((edge.token(), "Leaf"), [word])
        return leaf
    else:
        children = [wp_to_tree(t, wChart) for t in (edge.triple[1:])]
        lhs = Token(wChart._tokens[edge.start() : edge.end()],
                    edge.lhs(),
                    compute_semantics(children, edge))
        return Tree((lhs,edge.triple[0].__str__()), children)

In [ ]:
def bestTree(tokens, lex, rules):
    wChart = weightedParse(tokens, lex, rules)             # We build the weighgted parse tree using cky
    edge = list(wChart.select(start=0,end=len(tokens)))[0] # We get the biggest edge
    t = wp_to_tree(edge, wChart)                           # We get the tree that brought us to this edge
    return (t,edge.weight)

# Application of the algorithms and the grammar

## Weighed Lexicon

In [ ]:
from warnings import warn

class WeighedToken(Token):
    def __init__(self, token, categ, semantics=None, weight = 1.0):
        super().__init__(token, categ, semantics= semantics)
        self._weight = weight
    def weight(self):
        """1.0 is considered the default weight for any token"""
        try:
            return self._weight
        except AttributeError:
            warn(f"[{self.token} : {str(self)}] : this token has no weight attribute, defaulted to 1.0.")
            return 1.0

class WeighedLexicon(CCGLexicon):
    def __init__(self, start, primitives, families, entries):
        super().__init__(start, primitives, families, entries)

    def weight(self, entry):
        return entry.weight()

## Reading of the spreadsheet

In [ ]:
def to_pseudo_entries(table, consider_semantics = False, categories_max_options = 4):
    """returns a list of lists in the format ['word', 'category', 'weight', None]
    if consider_semantics == false else ['word', 'category', weight, 'semantic']
    that is left to be converted into tokens by to_wlex_entries"""

    entries = list()
    for line in range(len(table['MOT'])):
        for wdi, word in enumerate(table['MOT'][line].replace(" ", "").split('/')):
            for j in range(categories_max_options):
                if isinstance(table['Cat'+str(j)][line],str):
                    category = table['Cat'+str(j)][line]
                    weight = float(table['Weights'+str(j)][line]) if isinstance(table['Weights'+str(j)][line], Number) else 1.0
                    if consider_semantics:
                        semantic = (table['Sem'+str(j)][line].replace('\\\\', '\\').split('/'))[wdi]
                    else:
                        semantic = None
                    entries.append([word, category, weight, semantic])
    return entries

def to_wlex_entries(pseudo_entries, primitives, families, var=None):
    """returns the entries to a weighed lexicon from pseudo_entries generated by to_pseudo_entries"""
    entries = dict()
    for entry in pseudo_entries:
        if entry[0] not in entries:
            entries[entry[0]] = list()
        categ, _ = augParseCategory(entry[1], primitives, families, var)
        token = WeighedToken(token= entry[0],
                             categ= categ,
                             semantics= None if entry[-1] is None else Expression.fromstring(entry[-1]),
                             weight= entry[2])
        entries[entry[0]].append(token)
    return entries
    

## Instantiation of the lexicon

In [ ]:
# Catégories primitives et familles
primitives = ['S', 'N', 'Pp', 'pN']
V = augParseCategory("S\\N", primitives = primitives, families={})
families = {'V': V}

# On importe notre lexique sous forme de tableur
table = pd.read_excel("ccg.ods", engine="odf")
# On le convertit en Lexique pondéré
pe = to_pseudo_entries(table, consider_semantics = True)
wEntries = to_wlex_entries(pseudo_entries= pe, primitives= primitives, families= families)
lex = WeighedLexicon(start= 'S', primitives= primitives, families= families, entries= wEntries)


## Instantiation of the reduction rules

In [ ]:
# On crée le parser, on donne l'ensemble des règles qu'il est sensé connaître
rulesC  = [ForwardApplication,BackwardApplication] 
rulesC += [ForwardComposition,BackwardComposition,BackwardBx]
rulesC += [ForwardSubstitution,BackwardSx]
rulesR = [BinaryCombinatorRule(c) for c in rulesC]

parser = CCGChartParser(lex, rulesR)

On lit les phrases depuis le fichier `phrases.txt`, et pour chacune, on imprime le nombre de dérivations trouvées, ainsi que le meilleur arbre de dérivation (i.e. de meilleur poids)

## Reading test sentences

In [ ]:
# On lit les phrases dans le fichier
f = open('phrases.txt')
lines = f.readlines()
f.close()

phrases = [p.lower().strip().split() for p in lines]

## Running the above algorithms

In [ ]:
for tokens in phrases:
    
    print(reduce(lambda x,y: x + " " + y,tokens, ""))
    # On compte les arbres de dérivation trouvés
    try:
        i = len(list(parser.parse(tokens)))
    except:
            print("#SOME RANDOM ASSERT ERROR EVEN IF EVERYTHING WORKS FINE#")
    
    print("Found",i,"derivations for sentence",*tokens)

    # On affiche la dérivation la meilleure pour l'arbre
    if (i != 0):
        try:
            t,d = bestTree(tokens, lex, rulesC)
        except:
            print("#SOME RANDOM ASSERT ERROR EVEN IF EVERYTHING WORKS FINE#")
        print("Best derivation tree has weight",d)
        print('\n')
        printCCGDerivation(t)
    print('\n')
    print("#"*42)
    print('\n')

# Randomized testing

The code that we have used for testing strictness.

In [ ]:
def import_words(table):
    return list({word for line in range(len(table['MOT'])) for word in table['MOT'][line].replace(" ", "").split('/')})

def random_tests():
    Words = import_words(table)
    random_phrases = [reduce(lambda x,y: x + " " + y + " ", random.sample(Words, random.sample(range(2,11), 1)[0]), "") for i in range(500)]

    parsed = list()
    parses = dict()
    unparsed = list()
    for phr in random_phrases:
        try:
            t,d = bestTree(phr.split(), lex, rulesC)
            parsed.append(phr)
            parses[phr] = t
        except:
            unparsed.append(phr)

    print("="*50)
    print(f"found the following {len(parsed)} derivations:")
    for phr in parsed:
        print(phr + " :")
        printCCGDerivation(parses[phr])
    print("="*50)
    print(f'{len(unparsed)} are left unparsed :')
    for phr in unparsed:
        print(phr)

random_tests()

In [ ]:
print(lex)